In [145]:
import pandas as pd
import numpy as np
from pprint import pprint

In [176]:
dataset=pd.read_csv('new_data.csv')

In [178]:
dataset=dataset.drop(['Unnamed: 0','P. Composition Class','P. Habitable Class','P. Mass Class'],axis=1)

In [179]:
def entropy(target_col):
    elements,counts = np.unique(target_col,return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

In [180]:
def InfoGain(data,split_attribute_name,target_name="P. Zone Class"): 
    #Calculate the entropy of the total dataset
    total_entropy = entropy(data[target_name])
    #Calculate the values and the corresponding counts for the split attribute 
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    #Calculate the weighted entropy
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    #Calculate the information gain
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain

In [181]:
def ID3(data,originaldata,features,target_attribute_name="P. Zone Class",parent_node_class = None):
    
    #If all target_values have the same value, return this value
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    
    #If the dataset is empty, return the mode target feature value in the original dataset
    elif len(data)==0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]
    
    #If the feature space is empty, return the mode target feature value of the direct parent node --> Note that
    #the direct parent node is that node which has called the current run of the ID3 algorithm and hence
    #the mode target feature value is stored in the parent_node_class variable.
    
    elif len(features) ==0:
        return parent_node_class
    
    #If none of the above holds true, grow the tree!
    
    else:
        #Set the default value for this node --> The mode target feature value of the current node
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]
        
        #Select the feature which best splits the dataset
        item_values = [InfoGain(data,feature,target_attribute_name) for feature in features] #Return the information gain values for the features in the dataset
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        
        #Create the tree structure. The root gets the name of the feature (best_feature) with the maximum information
        #gain in the first run
        tree = {best_feature:{}}
        
        
        #Remove the feature with the best inforamtion gain from the feature space
        features = [i for i in features if i != best_feature]
        
        #Grow a branch under the root node for each possible value of the root node feature
        
        for value in np.unique(data[best_feature]):
            value = value
            #Split the dataset along the value of the feature with the largest information gain and therwith create sub_datasets
            sub_data = data.where(data[best_feature] == value).dropna()
            
            #Call the ID3 algorithm for each of those sub_datasets with the new parameters --> Here the recursion comes in!
            subtree = ID3(sub_data,dataset,features,target_attribute_name,parent_node_class)
            
            #Add the sub tree, grown from the sub_dataset to the tree under the root node
            tree[best_feature][value] = subtree
            
        return(tree)    
                

In [182]:
def predict(query,tree,default = 1):
    #1.
    for key in list(query.keys()):
        if key in list(tree.keys()):
            #2.
            try:
                result = tree[key][query[key]] 
            except:
                return default
  
            #3.
            result = tree[key][query[key]]
            #4.
            if isinstance(result,dict):
                return predict(query,result)
            else:
                return result

In [183]:
def train_test_split(dataset):
    training_data = dataset.iloc[:80].reset_index(drop=True)#We drop the index respectively relabel the index
    #starting form 0, because we do not want to run into errors regarding the row labels / indexes
    testing_data = dataset.iloc[80:].reset_index(drop=True)
    return training_data,testing_data
training_data = train_test_split(dataset)[0]
testing_data = train_test_split(dataset)[1] 
def test(data,tree):
    #Create new query instances by simply removing the target feature column from the original dataset and 
    #convert it to a dictionary
    queries = data.iloc[:,:-1].to_dict(orient = "records")
    
    #Create a empty DataFrame in whose columns the prediction of the tree are stored
    predicted = pd.DataFrame(columns=["predicted"]) 
    
    #Calculate the prediction accuracy
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree,1.0) 
    print('The prediction accuracy is: ',(np.sum(predicted["predicted"] == data["P. Zone Class"])/len(data))*100,'%')
    

'\nTrain the tree, Print the tree and predict the accuracy\n'

In [184]:
tree = ID3(training_data,training_data,training_data.columns[:-1])
pprint(tree)
test(testing_data,tree)

{'P. Name': {'11 Com b': 2.0,
             '11 Oph b': 1.0,
             '11 UMi b': 2.0,
             '14 And b': 2.0,
             '14 Her b': 1.0,
             '16 Cyg B b': 1.0,
             '18 Del b': 2.0,
             '1RXS 1609 b': 1.0,
             '1SWASP J1407 b': 1.0,
             '24 Boo b': 2.0,
             '24 Sex b': 2.0,
             '24 Sex c': 2.0,
             '2M 0103-55(AB) b': 1.0,
             '2M 0122-24 b': 1.0,
             '2M 0219-39 b': 1.0,
             '2M 0441+23 b': 1.0,
             '2M 0746+20 b': 1.0,
             '2M 1207-39 b': 1.0,
             '2M 1938+46 b': 1.0,
             '2M 2140+16 b': 1.0,
             '2M 2206-20 b': 1.0,
             '2M 2236+4751 b': 1.0,
             '2M J2126-81 b': 1.0,
             '2MASS J11193254 AB b': 0.0,
             '2MASS J1450-7841 Ab': 1.0,
             '2MASS J1450-7841 Bb': 1.0,
             '2MASS J2250+2325 b': 0.0,
             '30 Ari B b': 2.0,
             '38 Vir b': 3.0,
             '4 Uma b'